# Phi-3.5

In [8]:
! python -m pip install --upgrade pip
! pip install -r phi_requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [11]:
import os
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import json


# Open log file
log_path = './results/new_prompt/phi/'
os.makedirs(log_path, exist_ok=True)
log_file = os.path.join(log_path, 'prediction_log.txt')
log_file = open(log_file, 'w')

# Load data
df_table_prd = pd.read_csv('../LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('../LMM_sewerML/results/df_table_dsc.csv')
image_dir = '../LMM_sewerML/images'

# Load model and processor
model_id = "microsoft/Phi-3.5-vision-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    _attn_implementation='eager'
)
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
    num_crops=4  # Setting for single-frame performance
)

# Define the prompt template
prompt_template = """
You are a virtual sewer technician with the capability to analyze images from CCTV cameras taken inside sewer pipes.
Your task is to examine each image and provide a concise, yet accurate, summary for retrieval. 
After summarizing, you must classify the image as defect types.
While providing the summary, remember the following guidelines:"
1) Provide a general overview of the image that you see, describing important elements such image clarity, lighting conditions, type of pipe (concrete, PVC, ...), presence of water.
2) Check for defects in the sewer pipes in the image.
3) Pipes in good condition usually show a smooth, unbroken surface, no visible signs of damage like cracks or collapses, and an absence of blockages such as roots.
4) On the other hand, you can have the following defects:
4a)Cracks, Breaks, and Collapses (RB): Identify visible cracks along the pipe, instances where the pipe has fractured or completely broken apart, and areas where the pipe has collapsed.
This includes longitudinal cracks, circumferential breaks, and complete structural failures that compromise the integrity of the sewer system.
4b)Surface Damage (OB): Detect areas of the pipe's interior that exhibit signs of wear, erosion, or damage on the surface.
This includes minor scratches, pitting, scaling, or any form of deterioration that affects the pipe's surface but does not necessarily penetrate deeply into the structure.
4c) Production Error (PF): Identify defects that originated during the pipe's manufacturing process, such as inconsistent pipe thickness, improper joint alignment, or material imperfections.
These are flaws that were introduced before installation and could potentially affect the pipe's performance or longevity.
4d) Deformations (DE): Recognize any alterations in the shape of the pipe, such as bending, sagging, or bulging, that indicate a deformation.
This includes both minor deformations that may affect flow efficiency and major deformations that threaten the pipe's structural integrity.
4e) Roots (RO): Detect the presence of roots infiltrating the sewer pipe, whether through joints, cracks, or other vulnerabilities.
This involves identifying both the initial stages of root intrusion and the more advanced stages where roots have significantly obstructed the pipe.
5) Additional considerations while analyzing the images: do not consider blurred text or user-defined circled areas in the images.
6) You will always try to describe the image that you see. Provide the output in JSON format as follows:

{ "DESCRIPTION": "<Description of the image that you see>", "CODE": "<Defect Code>"}

Note: The "CODE" includes "RB", "OB", "PF", "DE", "RO". If no defect is detected, set "CODE" to "NoDefect".
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []


# Add tqdm progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=len(df_table_prd), desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        log_file.write(f"Image {img_id} not found. Skipping.\n")
        continue

    # Load image
    image = Image.open(img_path)
    images = [image]
    placeholder = "<|image_1|>\n"
    
    # Create prompt with image placeholder
    messages = [
        {"role": "user", "content": placeholder + prompt_template}
    ]
    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Prepare inputs
    inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")

    # Generate output
    generation_args = {
        "max_new_tokens": 1024,
        "temperature": 0.0,
        "do_sample": False,
    }
    with torch.inference_mode():
        generate_ids = model.generate(
            **inputs,
            eos_token_id=processor.tokenizer.eos_token_id,
            **generation_args
        )

        # Process output
        generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
        response = processor.batch_decode(
            generate_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        
    if response.startswith("```"):
        cleaned_response_text = response.split('\n', 1)[-1].rsplit('\n', 1)[0].strip()
    else:
        cleaned_response_text = response.strip()
    print(cleaned_response_text)
    # Extract defect prediction and description from model output
    try:
        # Attempt to parse the response as JSON
        response_json = json.loads(cleaned_response_text)
        description = response_json.get("DESCRIPTION", "No description provided")
        predicted_defect = response_json.get("CODE", "NoDefect")
    except json.JSONDecodeError:
        # Handle cases where the response is not valid JSON
        log_file.write(f"Invalid JSON response for Image ID {img_id}: {response}\n")
        description = "Error in JSON response"
        predicted_defect = "Error"

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(description)

    # Log intermediate outputs
    print()
    log_file.write(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

log_file.write(f"\nAccuracy: {accuracy:.4f}\n")
log_file.write(f"F1 Score: {f1:.4f}\n")
log_file.write("\nClassification Report:\n")
log_file.write(report)

# Close log file
log_file.close()

# Save results to CSV files
df_table_prd['phi-3.5'] = predictions
df_table_prd.to_csv(log_path + 'df_table_prd.csv', index=False)

df_table_dsc['phi-3.5'] = descriptions
df_table_dsc.to_csv(log_path + 'df_table_dsc.csv', index=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Processing images:   0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Processing images:   0%|          | 1/200 [00:03<11:57,  3.61s/it]

{
  "DESCRIPTION": "The image shows a section of a sewer pipe with a visible crack running vertically along its length. The surrounding area appears to be wet, suggesting the presence of water. The lighting conditions are dim, and the image is not very clear, making it difficult to discern finer details.",
  "CODE": "RB"
}
Image ID: 00617545.png | Predicted: RB | Ground Truth: RB



Processing images:   1%|          | 2/200 [00:09<15:57,  4.84s/it]

{

  "DESCRIPTION": "The image shows a section of a sewer pipe with a dark interior and a lighter, possibly metallic, exterior. The pipe appears to be in a state of disrepair with visible signs of corrosion and wear. There are no clear indications of water flow, and the lighting conditions are uneven, with some areas appearing brighter than others. The pipe's surface shows signs of scaling and pitting, suggesting long-term exposure to the elements or chemicals. No roots or significant deformations are visible, but the pipe's integrity is compromised due to the surface damage.",

  "CODE": "OB"

}
Image ID: 00635967.png | Predicted: OB | Ground Truth: NoDefect



...

Processing images: 100%|█████████▉| 199/200 [14:02<00:04,  4.15s/it]

{

  "DESCRIPTION": "The image shows a section of a sewer pipe with a yellowish hue, possibly due to the material or lighting conditions. The pipe appears to be made of concrete and is partially filled with water. There is a visible crack running vertically through the center of the pipe, which is a significant defect. No other defects such as roots, production errors, or deformations are visible in the image.",

  "CODE": "RB"

}
Image ID: 00835200.png | Predicted: RB | Ground Truth: DE



Processing images: 100%|██████████| 200/200 [14:06<00:00,  4.23s/it]

{

  "DESCRIPTION": "The image shows a sewer pipe with a smooth, unbroken surface and no visible signs of damage or blockages. The lighting conditions are even, and the pipe appears to be in good condition with no evidence of cracks, breaks, collapses, or deformations. There is no visible presence of roots or other obstructions.",

  "CODE": "NoDefect"

}
Image ID: 00507208.png | Predicted: NoDefect | Ground Truth: PF




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/